In [ ]:
import sys, traceback
from plantcv import plantcv as pcv
import cv2
import numpy as np
import glob
import os

In [ ]:

class options:
    def __init__(self):
        self.image = "../image_example/raspiU_cameraA.2021.09.07-20.30.01.jpg"
        self.debug = "plot"
        self.writeimg= True
        self.result = "./Results"
        self.outdir = "./Results"
        
# Store the output to the current directory 
# Get options
args = options()

# Set debug to the global parameter 
pcv.params.debug = args.debug

In [ ]:
#read in image
img, path, img_filename = pcv.readimage(filename=args.image, mode="native")

In [ ]:
# Corrects image based on color standard and stores output as corrected_img
corrected_img = pcv.white_balance(img=img, mode='hist', roi=[450, 1250, 100, 100])

In [ ]:
# Examine all colorspaces at one glance
colorspace_img = pcv.visualize.colorspaces(rgb_img=corrected_img)

In [ ]:
h = pcv.rgb2gray_hsv(rgb_img=corrected_img, channel='h')
s = pcv.rgb2gray_hsv(rgb_img=corrected_img, channel='s')
v = pcv.rgb2gray_hsv(rgb_img=corrected_img, channel='v')

In [ ]:
v_thresh = pcv.threshold.binary(gray_img=v, threshold=95, max_value=np.max(v),object_type='dark')

In [ ]:
l = pcv.rgb2gray_lab(rgb_img=corrected_img, channel='l')
a = pcv.rgb2gray_lab(rgb_img=corrected_img, channel='a')
b = pcv.rgb2gray_lab(rgb_img=corrected_img, channel='b')

In [ ]:
a_thresh1 = pcv.threshold.binary(gray_img=a, threshold=113, max_value=np.max(a), object_type='dark')

In [ ]:
# median filter on a_thresh image

a_thresh = pcv.median_blur(a_thresh1, 2)

In [ ]:
v_a = pcv.logical_or(bin_img1=v_thresh, bin_img2=a_thresh)

In [ ]:
masked = pcv.apply_mask(img=img, mask=v_a, mask_color="white")

In [ ]:
id_objects, obj_hierarchy = pcv.find_objects(img=masked, mask=a_thresh)

In [ ]:
roi1, roi_hierarchy = pcv.roi.rectangle(img= masked, x=350, y=200 , h=1300, w=1300)

In [ ]:
roi_objects, hierarchy3, kept_mask, obj_area = pcv.roi_objects(img = img, roi_contour=roi1,
                                                              roi_hierarchy=roi_hierarchy,
                                                              object_contour=id_objects,
                                                              obj_hierarchy= obj_hierarchy,
                                                              roi_type="partial")

In [ ]:
obj, mask = pcv.object_composition(img=img, contours=roi_objects, 
                                  hierarchy= hierarchy3)

In [ ]:
analysis_image = pcv.analyze_object(img=img, obj=obj, mask=mask)